In [8]:
import pandas as pd
df = pd.read_csv('../datasets/gabaeoh_data_from_hiDoc.csv')
df[:3]

,_id,question_title,question_content,question_date,question_tag,answer_title,answer_name,answer_belong,answer_content
0,64fece172822f039eb37dab3,강직성 척추염,선생님.\n\n강직성 척추염 환자의 90~95%에서 관련 유전자가 검출된다고 하셨는...,2016.03.18,"SMART TAG : 근골격계, 뼈, 정신/신경계, 내과, 류마티스내과, 강직성척추염",Re : 강직성 척추염,이정찬 [전문의],서울조인트내과의원,"안녕하세요, 하이닥 류마티스내과 상담의 이정찬 입니다.\n\n강직성 척추염의 증상이..."
1,64fece172822f039eb37dab4,척추골절 수술 문의드립니다.,60대 여성입니다.\n얼마전 허리상해로 mri 를 찍었는데\n병원 주치의 설명으로 ...,2023.05.23,"SMART TAG : 근골격계, 뼈, 척추, 정형외과, 재활의학과, 골절",Re : 척추골절 수술 문의드립니다.,이이호 [전문의],진주고려병원,질문자의 감사 인사 | 2023.05.23\n많은 도움이 되었습니다. 고맙습니다.
2,64fece172822f039eb37dab5,척추골절 수술 문의드립니다.,60대 여성입니다.\n얼마전 허리상해로 mri 를 찍었는데\n병원 주치의 설명으로 ...,2023.05.23,"SMART TAG : 근골격계, 뼈, 척추, 정형외과, 재활의학과, 골절",Re : 척추골절 수술 문의드립니다.,이이호 [전문의],진주고려병원,질문자의 감사 인사 | 2023.05.23\n많은 도움이 되었습니다. 고맙습니다.


In [10]:
df_question = df[['question_content', 'question_date','question_tag']]

In [17]:
df_question.to_excel('../datasets/select_colums.xlsx')

## 전처리 

### 결측치 처리

In [11]:
df_question.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10570 entries, 0 to 10569
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   question_content  10283 non-null  object
 1   question_date     10570 non-null  object
 2   question_tag      10561 non-null  object
dtypes: object(3)
memory usage: 247.9+ KB


In [13]:
df_question = df_question.dropna()
df_question.isnull().sum()

question_content    0
question_date       0
question_tag        0
dtype: int64

### 자연어 처리 
- 불용어 처리 
- 한글자 처리 

In [25]:
df_stop_words = pd.read_excel('../datasets/stop_words.xlsx')
df_stop_words[:5]

,불용어 리스트(stop word)
0,가
1,가까스로
2,가령
3,각
4,각각


In [23]:
#!pip install python-mecab-ko pyLDAvis

In [22]:
from mecab import MeCab
mecab = MeCab()

In [26]:
stopwords = df_stop_words['불용어 리스트(stop word)'].values
def tokenizer(raw, stopword=stopwords):
    result_list = list()
    for word in mecab.nouns(raw) :
        if len(word) > 1 and word not in stopword :
            result_list.append(word)
    return ' '.join(result_list) # return str

In [27]:
mecab.nouns('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.')

['곳', '운동', '수']

In [28]:
# 형태 분석기 적용된 함수 실행
tokenizer('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.')

'운동'

In [29]:
df_question['tokenized_content']= df_question['question_content'].apply(tokenizer)
df_question[:2]

,question_content,question_date,question_tag,tokenized_content
0,선생님.\n\n강직성 척추염 환자의 90~95%에서 관련 유전자가 검출된다고 하셨는...,2016.03.18,"SMART TAG : 근골격계, 뼈, 정신/신경계, 내과, 류마티스내과, 강직성척추염",선생 강직 척추염 환자 관련 유전자 검출 유전자 검출 사람 정상 확률 증상 경우 척...
1,60대 여성입니다.\n얼마전 허리상해로 mri 를 찍었는데\n병원 주치의 설명으로 ...,2023.05.23,"SMART TAG : 근골격계, 뼈, 척추, 정형외과, 재활의학과, 골절",여성 허리 병원 주치 설명 군데 골절 흉추 군데 요추 군데 압박 골절 통증 척추 성...


## 부정과 긍정 댓글 분리 